In [23]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16
from keras.optimizers import Adam

In [14]:
# Каталог с данными для обучения
train_dir = 'data/dogs_vs_cats/train'
# Каталог с данными для проверки
val_dir = 'data/dogs_vs_cats/val'
# Каталог с данными для тестирования
test_dir = 'data/dogs_vs_cats/test'

# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 16
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750

In [3]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=input_shape) # include_top - не будет загружена часть, отвечающая за классиф.

In [4]:
vgg16_net.trainable = False

In [5]:
vgg16_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [6]:
model = Sequential()
# Network vgg16 to model
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total para

In [8]:
model.compile(loss='binary_crossentropy',
             optimizer=Adam(lr=1e-5),
             metrics=['accuracy']) # low rate - низская скорость обучения для сходимости алгоритма

In [9]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [15]:
train_generator = datagen.flow_from_directory(
    train_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 17500 images belonging to 2 classes.


In [16]:
valid_generator = datagen.flow_from_directory(
    val_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3750 images belonging to 2 classes.


In [17]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3750 images belonging to 2 classes.


In [20]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=3,
    validation_data=valid_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/3
1093/1093 [==============================] - 3326s 3s/step - loss: 0.3975 - acc: 0.8173 - val_loss: 0.2989 - val_acc: 0.8782
Epoch 2/3
1093/1093 [==============================] - 3237s 3s/step - loss: 0.2919 - acc: 0.8764 - val_loss: 0.2607 - val_acc: 0.8948
Epoch 3/3
1093/1093 [==============================] - 3119s 3s/step - loss: 0.2588 - acc: 0.8905 - val_loss: 0.2448 - val_acc: 0.9004


In [21]:
model_json = model.to_json()
with open("vgg16_dogs_vs_cats_model.json", "w") as f:
    f.write(model_json)

model.save_weights("vgg16_dogs_vs_cats_model.h5")

In [24]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

KeyboardInterrupt: 

## Тонкая настройка сети (fine tuning)
"Размораживаем" последний сверточный блок сети VGG16

In [25]:
vgg16_net.trainable = True
trainable = False
for layer in vgg16_net.layers:
    if layer.name == 'block5_conv1':
        trainable = True
    layer.trainable = trainable

In [26]:
model.compile(loss='binary_crossentropy',
             optimizer=Adam(lr=1e-5),
             metrics=['accuracy']) # low rate - низская скорость обучения для сходимости алгоритма

In [27]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=2,
    validation_data=valid_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/2
   5/1093 [..............................] - ETA: 1:09:04 - loss: 0.1401 - acc: 0.9375

KeyboardInterrupt: 